In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#packages needed
#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
#the module 'sys' allows istalling module from inside Jupyter
import sys
import numpy as np 
import pandas as pd
#Natrual Language ToolKit (NLTK)
import nltk
from sklearn import metrics
#text normalization function
%run "/content/drive/MyDrive/Text_Normalization_Function.ipynb"
#ignore warnings about future changes in functions as they take too much space
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

     |████████████████████████████████| 49 kB 2.8 MB/s 
  Created wheel for html.parser: filename=html_parser-0.2-py3-none-any.whl size=1328 sha256=8695fba4c0589e456a72ce9b59fb3576192fb6511809c6aab73e871f4f1f763f
  Stored in directory: /root/.cache/pip/wheels/11/86/38/0554afea46105c70bae8d223c427bba371aa0c83ce88d57b27
Successfully built html.parser
     |████████████████████████████████| 23.7 MB 63.6 MB/s 
     |████████████████████████████████| 419 kB 38.2 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 81 kB 8.1 MB/s 
     |████████████████████████████████| 100 kB 10.3 MB/s 
     |████████████████████████████████| 130 kB 48.3 MB/s 
     |████████████████████████████████| 5.6 MB 32.1 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 3.6 MB 23.5 MB/s 
  Created wheel for pattern3: filename=pattern3-3.0.0-py2.py3-none-any.whl size=18554351 sha256=6391ee477e4e0b149c0e08d1f1093e023991

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping 

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/drive/MyDrive/reviews_Baby_5.json.gz')

In [67]:
df.new = df.loc[:,["reviewText","overall"]]

a = np.array(df.new.overall)
class_list = np.repeat("           ",len(a))
for i in range(len(a)):
    if a[i] == 5.0 or a[i] == 4.0:
        class_list[i]="High"
    if a[i] == 3.0:
        class_list[i] = "Medium"
    if a[i] == 2.0 or a[i] == 1.0:
        class_list[i] = "Low"
df.new["class"] = class_list

df.High = df.new.loc[df.new["class"]=="High"]
df.Medium = df.new.loc[df.new["class"]=="Medium"]
df.Low = df.new.loc[df.new["class"]=="Low"]

import random
random.seed(1)
High_train_index = random.sample(list(df.High.index),3350)
Medium_train_index = random.sample(list(df.Medium.index),3300)
Low_train_index = random.sample(list(df.Low.index),3350)

High_left = list(df.High.index)
for i in High_train_index:
    High_left.remove(i)

Medium_left = list(df.Medium.index)
for i in Medium_train_index:
    Medium_left.remove(i)

Low_left = list(df.Low.index)
for i in Low_train_index:
    Low_left.remove(i)
    
## Get the random index for the test data index; Total test data has 1,000 reviews with each 250
High_test_index = random.sample(High_left,335)
Medium_test_index = random.sample(Medium_left,353)
Low_test_index = random.sample(Low_left,335)


df_train = df.new.loc[High_train_index].append(df.new.loc[Medium_train_index],ignore_index=True).append(df.new.loc[Low_train_index],ignore_index= True)

df_test = df.new.loc[High_test_index].append(df.new.loc[Medium_test_index],ignore_index = True).append(df.new.loc[Low_test_index], ignore_index = True)

text_train = np.array(df_train["reviewText"])
class_train = np.array(df_train["class"])
text_test = np.array(df_test["reviewText"])
class_test = np.array(df_test["class"])

x_train_norm = normalize_corpus(text_train)
x_test_norm = normalize_corpus(text_test)
y_train = class_train
y_test = class_test

### Classification

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

bow_vectorizer = CountVectorizer()

BOW_matrix = bow_vectorizer.fit_transform(x_train_norm).toarray()
#pd.DataFrame(np.round(BOW_matrix,2))

x_train_bow = bow_vectorizer.fit_transform(x_train_norm)
corpus_train_bow_table = pd.DataFrame(data = x_train_bow.todense(), columns = bow_vectorizer.get_feature_names())
#corpus_train_bow_table.head(5)



In [7]:
## Select best 500 features
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

chi2_kbest = SelectKBest(score_func = chi2, k =500)
x_train_BEST = chi2_kbest.fit_transform(x_train_bow,y_train)

chi2_best_features_ind = chi2_kbest.get_support(indices=True)
chi2_best_features_names = np.array(bow_vectorizer.get_feature_names())[chi2_best_features_ind]

## Best features name
chi2_best_features_names

array(['3ml', 'absolutely', 'absorption', 'added', 'adhesive',
       'allowable', 'already', 'although', 'amazon', 'ameda', 'angle',
       'annoy', 'annoying', 'answer', 'anything', 'apart', 'apy',
       'armrest', 'asap', 'aspirator', 'attachment', 'attempt', 'awesome',
       'awful', 'awkward', 'bad', 'badly', 'bag', 'barely', 'basically',
       'basket', 'bassinet', 'battery', 'bebepod', 'beep', 'belt', 'best',
       'big', 'bird', 'bit', 'blanket', 'blinking', 'bonus', 'book',
       'booster', 'bottom', 'bra', 'bracket', 'break', 'britax', 'broken',
       'brush', 'buckle', 'button', 'buy', 'cable', 'call', 'called',
       'camera', 'canopy', 'cap', 'car', 'carrier', 'carry', 'cause',
       'chair', 'change', 'charge', 'charm', 'cheap', 'cheaply', 'cheapy',
       'child', 'classic', 'clay', 'clearly', 'color', 'company',
       'compartment', 'complaint', 'completely', 'con', 'concept', 'cons',
       'constantly', 'contact', 'cook', 'cooking', 'cosi', 'could',
       'c

In [8]:
x_train_bow_chi2_BEST_table = pd.DataFrame(data = x_train_BEST.todense(), columns = chi2_best_features_names)
#x_train_bow_chi2_BEST_table

In [68]:
NORM_x_test_bow = bow_vectorizer.transform(x_test_norm)
NORM_x_test_bow_chi2_BEST = chi2_kbest.transform(NORM_x_test_bow)
NORM_x_test_bow_chi2_BEST.shape
target_names = ["High","Medium",'Low']

## Naive Bayesian
from sklearn.naive_bayes import MultinomialNB
NB_tc = MultinomialNB(alpha=0.1) 
NB_tc.fit(x_train_BEST, y_train)

predicted_nb_chi2_best = NB_tc.predict(NORM_x_test_bow_chi2_BEST)  ## predictions
cm_chi2_best = metrics.confusion_matrix(y_test, predicted_nb_chi2_best)
print("Confusion matrix: \n", pd.DataFrame(data = cm_chi2_best , 
                                           columns = target_names,
                                           index = target_names),"\n")

print("Accuracy rate: ", metrics.accuracy_score(y_test, predicted_nb_chi2_best),"\n")    



from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predicted_nb_chi2_best)

Confusion matrix: 
         High  Medium  Low
High     210      38   87
Medium    39     233   63
Low       57     148  148 

Accuracy rate:  0.5777126099706745 



array([[210,  38,  87],
       [ 39, 233,  63],
       [ 57, 148, 148]])

In [69]:
## Tree Method
from sklearn.tree import DecisionTreeClassifier

Classification_tree = DecisionTreeClassifier()
Classification_tree.fit(x_train_BEST, y_train)

predicted_tree = Classification_tree.predict(NORM_x_test_bow_chi2_BEST) 
cm_chi2_best_tree = metrics.confusion_matrix(y_test, predicted_tree)
print("Confusion matrix: \n", pd.DataFrame(data = cm_chi2_best_tree , 
                                           columns = target_names,
                                           index = target_names),"\n")

print("Accuracy rate: ", metrics.accuracy_score(y_test, predicted_tree),"\n")    

Confusion matrix: 
         High  Medium  Low
High     166      68  101
Medium    61     173  101
Low       76     138  139 

Accuracy rate:  0.46725317693059626 



In [70]:
## Random Forest

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train_BEST,y_train)

pred_rf=clf.predict(NORM_x_test_bow_chi2_BEST)

cm_chi2_best_randomf = metrics.confusion_matrix(y_test, pred_rf)
print("Confusion matrix: \n", pd.DataFrame(data = cm_chi2_best_randomf , 
                                           columns = target_names,
                                           index = target_names),"\n")

print("Accuracy rate: ", metrics.accuracy_score(y_test, pred_rf),"\n")  

Confusion matrix: 
         High  Medium  Low
High     203      36   96
Medium    33     245   57
Low       61     148  144 

Accuracy rate:  0.5786901270772239 



In [71]:
## SVM 

from sklearn import linear_model

svm = linear_model.SGDClassifier(loss='hinge', random_state = 0) 
svm.fit(x_train_BEST, y_train)
predicted_svm = svm.predict(NORM_x_test_bow_chi2_BEST) 
cm_chi2_best_svm = metrics.confusion_matrix(y_test, predicted_svm)
print("Confusion matrix: \n", pd.DataFrame(data = cm_chi2_best_svm , 
                                           columns = target_names,
                                           index = target_names),"\n")

print("Accuracy rate: ", metrics.accuracy_score(y_test, predicted_svm),"\n")  

Confusion matrix: 
         High  Medium  Low
High     210      35   90
Medium    35     223   77
Low       56     145  152 

Accuracy rate:  0.5718475073313783 



In [72]:
## RNN 

from keras.layers import SimpleRNN
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))  # This last layer only returns the last outputs.
model.summary()

X_train=df_train.reviewText.str.strip('=').to_list()
Y_train=pd.get_dummies(df_train["class"]).values
X_test = df_test.reviewText.str.strip("=").to_list()
Y_test = pd.get_dummies(df_test["class"]).values


from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

# Number of words to consider as features
max_features = 1000
# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 40
batch_size = 10

#Tokenizing
tokenizer=Tokenizer(max_features,oov_token="<?>")
tokenizer.fit_on_texts(X_train)
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
print('Pad sequences (samples x time)')
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, None, 32)          2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, None, 32)          2080      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                2080      
                                                                 
Total params: 328,320
Trainable params: 328,320
Non-trainable params: 0
_________________________________________________________________
10000 train sequences
1023 test sequences
Pad se

In [73]:
from keras.layers import Dense
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(Y_test.shape[1], activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
#history = model.fit(X_train, Y_train,
                    #epochs=20,
                    #batch_size=batch_size,
                    #validation_data=(X_test,Y_test))
model.summary()
                

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 32)          32000     
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
Total params: 40,419
Trainable params: 40,419
Non-trainable params: 0
_________________________________________________________________


In [74]:
from keras import backend as K
K.clear_session()
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras import layers
from keras.models import Sequential

model = Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(Y_test.shape[1], activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=20, batch_size=10, validation_data = (X_test,Y_test))

Epoch 1/20
1000/1000 [==============================] - 27s 23ms/step - loss: 0.9660 - acc: 0.5003 - val_loss: 0.9416 - val_acc: 0.5464
Epoch 2/20
1000/1000 [==============================] - 22s 22ms/step - loss: 0.8461 - acc: 0.6063 - val_loss: 0.8612 - val_acc: 0.6061
Epoch 3/20
1000/1000 [==============================] - 22s 22ms/step - loss: 0.8048 - acc: 0.6347 - val_loss: 0.8629 - val_acc: 0.6041
Epoch 4/20
1000/1000 [==============================] - 22s 22ms/step - loss: 0.7813 - acc: 0.6474 - val_loss: 0.8630 - val_acc: 0.6168
Epoch 5/20
1000/1000 [==============================] - 23s 23ms/step - loss: 0.7619 - acc: 0.6620 - val_loss: 0.8607 - val_acc: 0.6119
Epoch 6/20
1000/1000 [==============================] - 22s 22ms/step - loss: 0.7387 - acc: 0.6726 - val_loss: 0.8786 - val_acc: 0.6188
Epoch 7/20
1000/1000 [==============================] - 22s 22ms/step - loss: 0.7171 - acc: 0.6897 - val_loss: 0.8709 - val_acc: 0.6031
Epoch 8/20
1000/1000 [==========================

In [75]:
## Convoluational 1D(CNN)

from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dropout

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Conv1D(32, 7,activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(Dropout(0.3))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(Y_test.shape[1], activation='softmax'))

model.summary()

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])
history = model.fit(X_train, Y_train,
                    epochs=20, # Originally 10, changed to 20 
                    batch_size=10,
                    validation_data = (X_test,Y_test))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 128)           128000    
                                                                 
 conv1d (Conv1D)             (None, 34, 32)            28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 11, 32)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 32)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 5, 32)             7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                       

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


1000/1000 [==============================] - 13s 7ms/step - loss: 1.0952 - acc: 0.3677 - val_loss: 1.0951 - val_acc: 0.3666
Epoch 2/20
1000/1000 [==============================] - 6s 6ms/step - loss: 1.0809 - acc: 0.4270 - val_loss: 1.0804 - val_acc: 0.4223
Epoch 3/20
1000/1000 [==============================] - 7s 7ms/step - loss: 1.0399 - acc: 0.4856 - val_loss: 1.0306 - val_acc: 0.4663
Epoch 4/20
1000/1000 [==============================] - 6s 6ms/step - loss: 0.9684 - acc: 0.5383 - val_loss: 0.9727 - val_acc: 0.5367
Epoch 5/20
1000/1000 [==============================] - 6s 6ms/step - loss: 0.9121 - acc: 0.5690 - val_loss: 0.9409 - val_acc: 0.5474
Epoch 6/20
1000/1000 [==============================] - 7s 7ms/step - loss: 0.8752 - acc: 0.5931 - val_loss: 0.9203 - val_acc: 0.5767
Epoch 7/20
1000/1000 [==============================] - 6s 6ms/step - loss: 0.8489 - acc: 0.6092 - val_loss: 0.9238 - val_acc: 0.5728
Epoch 8/20
1000/1000 [==============================] - 6s 6ms/step - lo

### Topic Modeling

In [16]:
#the module 'sys' allows istalling module from inside Jupyter
import sys

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install pandas
import pandas as pd

#Natrual Language ToolKit (NLTK)
!{sys.executable} -m pip install nltk
import nltk

!{sys.executable} -m pip install sklearn
from sklearn import metrics
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import  CountVectorizer #bag-of-words vectorizer 
from sklearn.decomposition import LatentDirichletAllocation #package for LDA

# Plotting tools

from pprint import pprint
!{sys.executable} -m pip install pyLDAvis #visualizing LDA
import pyLDAvis
import pyLDAvis.sklearn

import matplotlib.pyplot as plt
%matplotlib inline

#define text normalization function
%run ./Text_Normalization_Function.ipynb #defining text normalization function

#ignore warnings about future changes in functions as they take too much space
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

ERROR:root:File `'./Text_Normalization_Function.ipynb.py'` not found.


In [17]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
def get_topic_words(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_words = []
    for topic_weights in lda_model.components_:
        top_word_locs = (-topic_weights).argsort()[:n_words]
        topic_words.append(keywords.take(top_word_locs).tolist())
    return topic_words

In [18]:
#define the bag-of-words vectorizer:
bow_vectorizer = CountVectorizer()

#vectorize the normalized data:
bow_toy_corpus = bow_vectorizer.fit_transform(x_train_norm)

In [20]:
!{sys.executable} -m pip install gensim
import gensim

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [21]:
bow_vectorizer = CountVectorizer()
bow_corpus = bow_vectorizer.fit_transform(x_train_norm)

# LDA 2-Topic

In [22]:
lda_corpus_2 = LatentDirichletAllocation(n_components=2, max_iter=500,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.9).fit(bow_toy_corpus)

In [23]:
no_top_words = 15
display_topics(lda_corpus_2, bow_vectorizer.get_feature_names(), no_top_words)

Topic 0:
baby use like love month diaper buy little great really well old good toy time
Topic 1:
use seat bottle like work baby easy cup stroller time car well great good buy


In [26]:
word_weights_2 = lda_corpus_2.components_ / lda_corpus_2.components_.sum(axis=1)[:, np.newaxis]
print(pd.DataFrame(word_weights_2.T, index = bow_vectorizer.get_feature_names()).T)

      000sf      00am   06scion  ...    zurich     zzzzs  생각되어bib을
0  0.000004  0.000004  0.000004  ...  0.000008  0.000007  0.000004
1  0.000008  0.000008  0.000008  ...  0.000004  0.000004  0.000008

[2 rows x 15386 columns]


In [27]:
#tokenizing the corpus
news_corpus_tokenized = [tokenize_text(x_train_norm[doc_id]) for doc_id in range(len(x_train_norm))]

#Dictionary of the corpus:
news_dictionary = Dictionary(news_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
news_corpus_bow = [news_dictionary.doc2bow(doc) for doc in news_corpus_tokenized]

#top 20 words for each topic (using the function defined in session prep)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer, lda_model = lda_corpus_2, n_words=20)

In [28]:
cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 2))  # get coherence value

Coherence score for the model:  -1.49


In [29]:
print("Coherence score by topic (higher values are better): ", np.round(cm.get_coherence_per_topic(),2))

Coherence score by topic (higher values are better):  [-1.49 -1.5 ]


In [30]:
print("Log-Likelihood (higher values are better): ", lda_corpus_2.score(bow_corpus))

Log-Likelihood (higher values are better):  -3315730.103538731


In [31]:
print("Perplexity (lower values are better): ", lda_corpus_2.perplexity(bow_corpus))

Perplexity (lower values are better):  1599.7215725335789


# LDA 3-Topic

In [32]:
lda_corpus_3 = LatentDirichletAllocation(n_components=3, max_iter=500,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.9).fit(bow_toy_corpus)

In [33]:
no_top_words = 15
display_topics(lda_corpus_3, bow_vectorizer.get_feature_names(), no_top_words)

Topic 0:
use baby diaper bag like well buy fit good cover wash great soft crib size
Topic 1:
seat baby like use love toy old stroller month car easy little son great really
Topic 2:
use bottle baby work like cup time pump great buy good water well really monitor


In [37]:
word_weights_3 = lda_corpus_3.components_ / lda_corpus_3.components_.sum(axis=1)[:, np.newaxis]
print(pd.DataFrame(word_weights_3.T, index = bow_vectorizer.get_feature_names()).T)

      000sf      00am   06scion  ...    zurich     zzzzs  생각되어bib을
0  0.000006  0.000006  0.000007  ...  0.000013  0.000007  0.000012
1  0.000005  0.000005  0.000009  ...  0.000005  0.000009  0.000005
2  0.000012  0.000012  0.000006  ...  0.000006  0.000006  0.000007

[3 rows x 15386 columns]


In [39]:
#tokenizing the corpus
news_corpus_tokenized = [tokenize_text(x_train_norm[doc_id]) for doc_id in range(len(x_train_norm))]

#Dictionary of the corpus:
news_dictionary = Dictionary(news_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
news_corpus_bow = [news_dictionary.doc2bow(doc) for doc in news_corpus_tokenized]

#top 20 words for each topic (using the function defined in session prep)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer, lda_model = lda_corpus_3, n_words=20)

In [40]:
cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 3))  # get coherence value

Coherence score for the model:  -1.68


In [41]:
print("Coherence score by topic (higher values are better): ", np.round(cm.get_coherence_per_topic(),3))

Coherence score by topic (higher values are better):  [-1.76  -1.436 -1.845]


In [42]:
print("Log-Likelihood (higher values are better): ", lda_corpus_3.score(bow_corpus))

Log-Likelihood (higher values are better):  -3321837.523717019


In [43]:
print("Perplexity (lower values are better): ", lda_corpus_3.perplexity(bow_corpus))

Perplexity (lower values are better):  1621.608834455664


# LDA 4-Topic

In [44]:
lda_corpus_4 = LatentDirichletAllocation(n_components=4, max_iter=500,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.9).fit(bow_toy_corpus)

In [45]:
no_top_words = 15
display_topics(lda_corpus_4, bow_vectorizer.get_feature_names(), no_top_words)

Topic 0:
baby use work sleep time monitor product night light room sound like swing great pillow
Topic 1:
seat use stroller car easy like baby put fit strap well chair need old child
Topic 2:
baby love like toy month use old little son buy well really good great soft
Topic 3:
use bottle diaper bag cup like work time pump baby clean buy well good water


In [47]:
word_weights_4 = lda_corpus_4.components_ / lda_corpus_4.components_.sum(axis=1)[:, np.newaxis]
print(pd.DataFrame(word_weights_4.T, index = bow_vectorizer.get_feature_names()).T)

      000sf      00am   06scion  ...    zurich     zzzzs  생각되어bib을
0  0.000020  0.000019  0.000009  ...  0.000012  0.000018  0.000017
1  0.000006  0.000006  0.000013  ...  0.000007  0.000007  0.000007
2  0.000007  0.000007  0.000007  ...  0.000009  0.000007  0.000007
3  0.000007  0.000007  0.000007  ...  0.000009  0.000007  0.000007

[4 rows x 15386 columns]


In [49]:
#tokenizing the corpus
news_corpus_tokenized = [tokenize_text(x_train_norm[doc_id]) for doc_id in range(len(x_train_norm))]

#Dictionary of the corpus:
news_dictionary = Dictionary(news_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
news_corpus_bow = [news_dictionary.doc2bow(doc) for doc in news_corpus_tokenized]

#top 20 words for each topic (using the function defined in session prep)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer, lda_model = lda_corpus_4, n_words=20)

In [50]:
cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 4))  # get coherence value

Coherence score for the model:  -1.7321


In [52]:
Coherence score by topic (higher values are better):  [-1.6708 -1.7848 -1.5817 -1.9948]

SyntaxError: ignored

In [53]:
print("Log-Likelihood (higher values are better): ", lda_corpus_4.score(bow_corpus))

Log-Likelihood (higher values are better):  -3334187.0675827833


In [54]:
print("Perplexity (lower values are better): ", lda_corpus_4.perplexity(bow_corpus))

Perplexity (lower values are better):  1666.7853267598312


# LDA 5-Topic

In [55]:
lda_corpus_5 = LatentDirichletAllocation(n_components=5, max_iter=500,
                                           doc_topic_prior = 0.9,
                                           topic_word_prior = 0.9).fit(bow_toy_corpus)

In [56]:
no_top_words = 15
display_topics(lda_corpus_5, bow_vectorizer.get_feature_names(), no_top_words)

Topic 0:
bottle use baby work pump time monitor nipple milk like product good buy need much
Topic 1:
baby love month toy like old son play little really great daughter buy color cute
Topic 2:
use cup clean like easy work little well water food open put plastic hold great
Topic 3:
use diaper bag baby fit like wash cover size well good buy crib soft great
Topic 4:
seat stroller car use baby like easy strap chair fit child well put take great


In [61]:
word_weights_5 = lda_corpus_5.components_ / lda_corpus_5.components_.sum(axis=1)[:, np.newaxis]
print(pd.DataFrame(word_weights_5.T, index = bow_vectorizer.get_feature_names()).T)

      000sf      00am   06scion  ...    zurich     zzzzs  생각되어bib을
0  0.000019  0.000019  0.000009  ...  0.000010  0.000010  0.000012
1  0.000010  0.000010  0.000009  ...  0.000010  0.000017  0.000012
2  0.000009  0.000009  0.000009  ...  0.000009  0.000009  0.000011
3  0.000008  0.000008  0.000008  ...  0.000016  0.000008  0.000010
4  0.000008  0.000008  0.000016  ...  0.000008  0.000009  0.000009

[5 rows x 15386 columns]


In [62]:
#tokenizing the corpus
news_corpus_tokenized = [tokenize_text(x_train_norm[doc_id]) for doc_id in range(len(x_train_norm))]

#Dictionary of the corpus:
news_dictionary = Dictionary(news_corpus_tokenized)

#Bag-of-words representation for each document of the corpus:
news_corpus_bow = [news_dictionary.doc2bow(doc) for doc in news_corpus_tokenized]

#top 20 words for each topic (using the function defined in session prep)
topic_topwords = get_topic_words(vectorizer = bow_vectorizer, lda_model = lda_corpus_5, n_words=20)

In [63]:
cm = CoherenceModel(topics=topic_topwords, 
                    corpus = news_corpus_bow , 
                    dictionary = news_dictionary, coherence='u_mass')
print("Coherence score for the model: ", np.round(cm.get_coherence(), 5))  # get coherence value

Coherence score for the model:  -1.83239


In [64]:
print("Coherence score by topic (higher values are better): ", np.round(cm.get_coherence_per_topic(),5))

Coherence score by topic (higher values are better):  [-1.70601 -1.78695 -1.96551 -2.08428 -1.61919]


In [65]:
print("Log-Likelihood (higher values are better): ", lda_corpus_5.score(bow_corpus))

Log-Likelihood (higher values are better):  -3351141.091209992


In [66]:
print("Perplexity (lower values are better): ", lda_corpus_5.perplexity(bow_corpus))

Perplexity (lower values are better):  1730.8627030607724


### Sentiment Analysis

In [76]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [77]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [78]:
def analyze_sentiment_vader_lexicon(review, threshold1 = -0.3,threshold2 = 0, verbose = False):
    scores = analyzer.polarity_scores(review)
    if scores['compound'] <= threshold1:
        human_sentiment = 'Low'
    elif scores['compound'] <= threshold2:
        human_sentiment = 'Medium'
    else:
        human_sentiment = 'High'
    if verbose:                             
        print('VADER Polarity (human):', human_sentiment)
        print('VADER Score:', round(scores['compound'], 2))
    return human_sentiment,scores['compound']  

In [79]:
VADER_polarity_test = [analyze_sentiment_vader_lexicon(review, threshold1 = 0.5,threshold2 = 0.8) for review in x_test_norm]
VADER_polarity_test_df = pd.DataFrame(VADER_polarity_test, columns = ['VADER Polarity','VADER Score'])
print(VADER_polarity_test_df.head(30))

   VADER Polarity  VADER Score
0            High       0.9323
1            High       0.8860
2             Low       0.3612
3            High       0.9543
4            High       0.9868
5            High       0.8126
6            High       0.9584
7            High       0.9022
8            High       0.9042
9            High       0.9042
10           High       0.8957
11            Low       0.1531
12           High       0.8271
13           High       0.9764
14           High       0.8839
15           High       0.9630
16            Low       0.3818
17           High       0.9595
18           High       0.9716
19           High       0.8642
20            Low       0.4404
21           High       0.8316
22            Low      -0.2263
23         Medium       0.5367
24         Medium       0.7269
25         Medium       0.6901
26           High       0.9341
27            Low       0.1010
28           High       0.8126
29           High       0.8807


In [80]:
def format(value):
    value = str(value)
    return value
VADER_polarity_test_df['VADER Polarity'] = VADER_polarity_test_df['VADER Polarity'].apply(format)

In [81]:
from sklearn import metrics
print('Accuracy Rate:', np.round(metrics.accuracy_score(class_test, VADER_polarity_test_df['VADER Polarity']), 3),"\n")

Accuracy Rate: 0.464 



In [82]:
cross_table = metrics.confusion_matrix(class_test, VADER_polarity_test_df['VADER Polarity'])
print("Confusion matrix: \n", pd.DataFrame(data = cross_table, columns = ["High", "Medium", "Low"], index = ["High", "Medium", "Low"]),"\n")

Confusion matrix: 
         High  Medium  Low
High     236      39   60
Medium   114     146   75
Low      182      78   93 

